In [1]:
import os
import pandas as pd
import qiime2 as q2
from qiime2 import Visualization

%matplotlib inline

In [2]:
data = 'Data'

In [3]:
data_classified = 'Data/classified'

In [4]:
data_raw = 'Data/raw'

In [5]:
data_alpha = 'Data/diversity'

In [23]:
! qiime diversity alpha-rarefaction \
    --i-table $data_classified/table-filtered.qza \
    --p-max-depth 10000 \
    --p-iterations 10 \
    --m-metadata-file $data_raw/20250913_metadata_ITS.tsv \
    --o-visualization $data_alpha/alpha-rarefaction.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Data/diversity/alpha-rarefaction.qzv


In [6]:
Visualization.load(f"{data_alpha}/alpha-rarefaction.qzv")

<visualization: Visualization uuid: 1ec9a82b-7309-4564-9f43-c6e9e6d7ec18>

In [30]:
Visualization.load(f"{data_classified}/table-filtered.qzv")

<visualization: Visualization uuid: 3432396a-f90f-41b9-966a-254ce535c107>

### we chose a sampling depth of 3000 because it can retain 87% of the samples


***core metrics creation***

In [19]:
! qiime diversity core-metrics \
  --i-table $data_classified/table-filtered.qza \
  --m-metadata-file $data_raw/20250913_metadata_ITS.tsv \
  --p-sampling-depth 3000 \
  --output-dir $data_alpha/core-metrics-results

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: Data/diversity/core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: Data/diversity/core-metrics-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: Data/diversity/core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: Data/diversity/core-metrics-results/evenness_vector.qza
Saved DistanceMatrix to: Data/diversity/core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: Data/diversity/core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: Data/diversity/core-metrics-results/jaccard_pcoa_results.qza
Saved PCoAResults to: Data/dive

***Test for parametric data: 1) data normally distributed?***

In [7]:
#hier stehen geblieben!! Testen, ob normally distributed!
from scipy.stats import shapiro

shannon_path = f"{data_alpha}/core-metrics-results/shannon_vector.qza"
shannon = q2.Artifact.load(shannon_path).view(pd.Series)

stat, p = shapiro(shannon)
print('Statistics=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
    print("Daten wahrscheinlich normalverteilt")
else:
    print("Daten wahrscheinlich nicht normalverteilt")

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Statistics=0.934, p=0.000
Daten wahrscheinlich nicht normalverteilt


/opt/conda/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')


***Kruskal-Wallis: Non-parametric testing, because data not normally distributed***

In [20]:
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_alpha/core-metrics-results/shannon_vector.qza \
  --m-metadata-file $data_raw/20250913_metadata_ITS.tsv \
  --o-visualization $data_alpha/core-metrics-results/shannon-group-significance.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Data/diversity/core-metrics-results/shannon-group-significance.qzv


In [8]:
Visualization.load(f"{data_alpha}/core-metrics-results/shannon-group-significance.qzv")

<visualization: Visualization uuid: ca6d8ee4-2e1e-4118-8541-870d7c98d206>

In [ ]:
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_alpha/core-metrics-results/shannon_vector.qza \
  --m-metadata-file $data_raw/20250913_metadata_ITS.tsv \
  --o-visualization $data_alpha/core-metrics-results/shannon-group-significance-numeric.qzv

In [9]:
Visualization.load(f"{data_alpha}/core-metrics-results/shannon-group-significance-numeric.qzv")

<visualization: Visualization uuid: 15be04d1-0002-4fb2-8f21-a44956e689cd>